In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
import re
from sklearn.pipeline import Pipeline
import nltk
from nltk.stem.snowball import SnowballStemmer
from sklearn.metrics import f1_score
from nltk.corpus import stopwords
import pickle
import tweepy
from IPython.display import clear_output
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV

In [2]:
rf_df = pd.read_csv('data/full/rf_preproc_train.csv')

In [3]:
rf_df = rf_df[['user_id', 'avg_fav', 'avg_hash', 'avg_len', 'avg_ret', 'default_profile',
       'fake_words_score', 'favourites_count', 'followers_count', 'freq',
       'friends_count', 'genuine_words_score', 'listed_count', 'max_fav',
       'max_hash', 'max_len', 'max_ret', 'media_perc', 'min_fav', 'min_hash',
       'min_len', 'min_ret', 'porn_words_score',
       'profile_use_background_image', 'prop_words_score', 'quote_perc',
       'ret_perc', 'spam_words_score', 'statuses_count',
       'tweet_intradistance', 'url', 'url_intradistance', 'url_perc',
       'description_len', 'name_len', 'screen_name_len', 'age','target', 'nsfw_avg', 'nsfw_profile']]

In [4]:
X_train, X_val, y_train, y_val = train_test_split(rf_df.drop(columns=['target']), rf_df['target'], test_size=0.2, random_state=42)

In [5]:
rf = RandomForestClassifier(n_estimators=40, criterion='gini', max_depth=28)

In [6]:
cv = cross_validate(rf, X_train.drop(columns=['user_id']), y_train, cv=10, scoring='f1_macro')

In [7]:
cv['test_score'].mean()

0.9625573597485287

In [8]:
rf.fit(X_train.drop(columns=['user_id']), y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=28, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=40, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [9]:
rf_pred = rf.predict_proba(X_val.drop(columns=['user_id']))

In [10]:
rf.predict_proba(X_val.drop(columns=['user_id']).iloc[0:1])

array([[0.55 , 0.   , 0.   , 0.275, 0.175]])

In [11]:
rf_prob = pd.DataFrame(rf_pred, columns=['rf_0', 'rf_1', 'rf_2', 'rf_3', 'rf_4'])

In [12]:
X_val.reset_index(inplace=True, drop=True)

In [13]:
rf_prob.reset_index(inplace=True, drop=True)

In [14]:
y_val.reset_index(inplace=True, drop=True)

In [15]:
rf_prob['user_id'] = X_val['user_id']

In [16]:
rf_prob['target'] = y_val

In [17]:
rf_prob.head()

,rf_0,rf_1,rf_2,rf_3,rf_4,user_id,target
0,0.550,0.000,0.0,0.275,0.175,610392907,0
1,0.975,0.000,0.0,0.000,0.025,239615243,0
2,0.000,1.000,0.0,0.000,0.000,705441738391838720,1
3,0.000,0.925,0.0,0.000,0.075,32871086,1
4,1.000,0.000,0.0,0.000,0.000,185036273,0


## BotOrNot

In [18]:
model = pickle.load(open("data/bot_or_not/model.model", "rb"))

In [19]:
X_val.drop(columns=['porn_words_score', 'prop_words_score', 'spam_words_score', 'fake_words_score', 'genuine_words_score'], inplace=True)

In [20]:
y_bon = y_val

In [21]:
y_bon[:13]

0     0
1     0
2     1
3     1
4     0
5     0
6     0
7     2
8     1
9     3
10    1
11    0
12    4
Name: target, dtype: int64

In [22]:
y_bon[y_bon != 4] = 1
y_bon[y_bon == 4] = 0

In [23]:
X_val.drop(columns=['nsfw_profile', 'nsfw_avg'], inplace=True)
X_train.drop(columns=['nsfw_profile', 'nsfw_avg'], inplace=True)

In [24]:
bon_pred = model.predict_proba(X_val.drop(columns=['user_id']))

In [25]:
bon_pred

array([[0.17142857, 0.82857143],
       [0.02857143, 0.97142857],
       [0.        , 1.        ],
       ...,
       [0.14285714, 0.85714286],
       [0.08571429, 0.91428571],
       [1.        , 0.        ]])

In [26]:
bon_prob = pd.DataFrame(bon_pred, columns=['bon_4', 'bon_3'])

bon_prob['bon_3'] = bon_pred.astype(int)

bon_prob['bon_4'] = np.logical_not(bon_pred).astype(int)

In [27]:
bon_prob['bon_0'] = 0
bon_prob['bon_1'] = 0
bon_prob['bon_2'] = 0

In [28]:
y_pred = model.predict(X_val.drop(columns=['user_id']))

In [29]:
print(pd.crosstab(y_bon, y_pred , rownames=['Actual Target'], colnames=['Predicted Target']))

Predicted Target  0.0   1.0
Actual Target              
0                 696     8
1                 200  4118


In [30]:
bon_prob['bon_0'] = bon_prob['bon_3']/4
bon_prob['bon_1'] = bon_prob['bon_3']/4
bon_prob['bon_2'] = bon_prob['bon_3']/4
bon_prob['bon_3'] = bon_prob['bon_3']/4

In [31]:
X_val.reset_index(inplace=True, drop=True)

In [32]:
bon_prob.reset_index(inplace=True, drop=True)

In [33]:
bon_prob['user_id'] = X_val['user_id']

In [34]:
bon_prob['target'] = rf_prob['target']

In [35]:
bon_prob = bon_prob.reindex(columns=['bon_0', 'bon_1', 'bon_2', 'bon_3', 'bon_4', 'user_id', 'target'])

In [36]:
users = pd.read_csv('data/full/train.csv')

In [37]:
users = users[['user_id','target']]

In [38]:
tweets = pd.concat([pd.read_csv('data/propaganda/tweets.csv', sep='\t')[['user_id','full_text']], \
          pd.read_csv('data/porn/tweets.csv', sep='\t')[['user_id','full_text']], \
          pd.read_csv('data/spam/tweets.csv', sep='\t')[['user_id','full_text']], \
          pd.read_csv('data/fake_followers/tweets.csv', sep='\t')[['user_id','full_text']], \
          pd.read_csv('data/genuine/tweets.csv', sep='\t')[['user_id','full_text']]])

/home/matteo/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (1,8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/matteo/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (1,8,18) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/matteo/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (1,4,8,11,18,19,20,23,26) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/matteo/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (1,8,31,32,33) have mixed types. Specify dtype option on import or set low_memory=False.
  int

In [39]:
tweets = tweets.drop_duplicates()

In [40]:
tweets = pd.merge(tweets, users, on='user_id')

In [41]:
mask = np.in1d(tweets.user_id, X_train.user_id)

In [42]:
val_mask = np.in1d(tweets.user_id, X_val.user_id)

In [43]:
nb_train = tweets[mask]

In [44]:
nb_val = tweets[val_mask]

In [45]:
def remove_rt(x):
    if 'RT @' in x:
        try:
            return x[x.find(':')+2:]
        except:
            return x
    else:
        return x

In [46]:
stop_words = stopwords.words('english')

def remove_stop(x):
    return [word for word in x.split() if word not in stop_words]

In [47]:
nb_train['full_text'] = nb_train['full_text'].apply(lambda x: remove_rt(x))
nb_train['full_text'] = nb_train['full_text'].apply(lambda x: re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '', x))
nb_train['full_text'] = nb_train['full_text'].apply(lambda x: re.sub(r'[^\w\s]','',x))
nb_train['full_text'] = nb_train['full_text'].apply(lambda x: x.lower())
nb_train['full_text'] = nb_train['full_text'].apply(lambda x: remove_stop(x))
nb_train['full_text'] = nb_train['full_text'].astype(str)
nb_train = nb_train[nb_train['full_text']!='[]']

/home/matteo/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/matteo/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/matteo/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docum

In [48]:
nb_val['full_text'] = nb_val['full_text'].apply(lambda x: remove_rt(x))
nb_val['full_text'] = nb_val['full_text'].apply(lambda x: re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '', x))
nb_val['full_text'] = nb_val['full_text'].apply(lambda x: re.sub(r'[^\w\s]','',x))
nb_val['full_text'] = nb_val['full_text'].apply(lambda x: x.lower())
nb_val['full_text'] = nb_val['full_text'].apply(lambda x: remove_stop(x))
nb_val['full_text'] = nb_val['full_text'].astype(str)
nb_val = nb_val[nb_val['full_text']!='[]']

/home/matteo/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/matteo/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/matteo/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docum

In [49]:
stemmer = SnowballStemmer("english", ignore_stopwords=True)

class StemmedCountVectorizer(CountVectorizer):
    def build_analyzer(self):
        analyzer = super(CountVectorizer, self).build_analyzer()
        return lambda doc:(stemmer.stem(w) for w in analyzer(doc))


stem_vectorizer = StemmedCountVectorizer(stemmer)

pipeline = Pipeline([('vect', stem_vectorizer), 
                     ('tfidf', TfidfTransformer()), 
                     ('mnb', MultinomialNB(fit_prior=False))])

clf = pipeline.fit(nb_train['full_text'], nb_train['target'])

In [50]:
predictions = []
i = 0

for usr in X_val['user_id']:
    
    i+=1
    clear_output()
    print(i)
    tweets_list = nb_val[nb_val.user_id == usr]['full_text']
    tweets_list = tweets_list[tweets_list.notnull()]
    if len(tweets_list[tweets_list.notnull()]) != 0:
        pred = clf.predict_proba(tweets_list)
        predictions.append(np.mean(pred, axis=0))
    else:
        predictions.append(np.array([0.2,0.2,0.2,0.2,0.2]))

5022


In [51]:
prob = pd.DataFrame(predictions, columns=['nb_0', 'nb_1', 'nb_2', 'nb_3', 'nb_4'])

In [52]:
prob['target'] = rf_prob['target']

In [53]:
prob['user_id'] = X_val['user_id']

In [54]:
# prob.to_csv('data/full/predictions1.csv')

In [55]:
stack_df = rf_prob.merge(prob, on=['user_id' , 'target'])

In [56]:
stack_df = stack_df.merge(bon_prob, on=['user_id' , 'target'])

In [57]:
stack_df = stack_df.reindex_axis(sorted(stack_df.columns), axis=1)

/home/matteo/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: '.reindex_axis' is deprecated and will be removed in a future version. Use '.reindex' instead.
  """Entry point for launching an IPython kernel.


In [58]:
stack_df.head()

,bon_0,bon_1,bon_2,bon_3,bon_4,nb_0,nb_1,nb_2,nb_3,nb_4,rf_0,rf_1,rf_2,rf_3,rf_4,target,user_id
0,0.207143,0.207143,0.207143,0.207143,0.171429,0.200000,0.200000,0.200000,0.200000,0.200000,0.550,0.000,0.0,0.275,0.175,0,610392907
1,0.242857,0.242857,0.242857,0.242857,0.028571,0.200000,0.200000,0.200000,0.200000,0.200000,0.975,0.000,0.0,0.000,0.025,0,239615243
2,0.250000,0.250000,0.250000,0.250000,0.000000,0.022305,0.844103,0.065607,0.003753,0.064231,0.000,1.000,0.0,0.000,0.000,1,705441738391838720
3,0.121429,0.121429,0.121429,0.121429,0.514286,0.013702,0.883903,0.049563,0.000985,0.051848,0.000,0.925,0.0,0.000,0.075,1,32871086
4,0.250000,0.250000,0.250000,0.250000,0.000000,0.200000,0.200000,0.200000,0.200000,0.200000,1.000,0.000,0.0,0.000,0.000,0,185036273


In [59]:
stack_df.to_csv('data/full/logreg_train.csv')

In [60]:
stack_df

,bon_0,bon_1,bon_2,bon_3,bon_4,nb_0,nb_1,nb_2,nb_3,nb_4,rf_0,rf_1,rf_2,rf_3,rf_4,target,user_id
0,0.207143,0.207143,0.207143,0.207143,0.171429,0.200000,0.200000,0.200000,0.200000,0.200000,0.550,0.000,0.000,0.275,0.175,0,610392907
1,0.242857,0.242857,0.242857,0.242857,0.028571,0.200000,0.200000,0.200000,0.200000,0.200000,0.975,0.000,0.000,0.000,0.025,0,239615243
2,0.250000,0.250000,0.250000,0.250000,0.000000,0.022305,0.844103,0.065607,0.003753,0.064231,0.000,1.000,0.000,0.000,0.000,1,705441738391838720
3,0.121429,0.121429,0.121429,0.121429,0.514286,0.013702,0.883903,0.049563,0.000985,0.051848,0.000,0.925,0.000,0.000,0.075,1,32871086
4,0.250000,0.250000,0.250000,0.250000,0.000000,0.200000,0.200000,0.200000,0.200000,0.200000,1.000,0.000,0.000,0.000,0.000,0,185036273
5,0.135714,0.135714,0.135714,0.135714,0.457143,0.200000,0.200000,0.200000,0.200000,0.200000,1.000,0.000,0.000,0.000,0.000,0,977008152
6,0.178571,0.178571,0.178571,0.178571,0.285714,0.326498,0.145101,0.187060,0.109445,0.231896,0.975,0.000,0.025,0.000,0.000,0,1918781449
7,0.164286,0.164286,0.164286,0.164286,0.342857,0.056578,0.086992,0.692061,0.015394,0.148975,0.000,0.000,1.000,0.000,0.000,2,2348486173
8,0.200000,0.200000,0.200000,0.200000,0.200000,0.034877,0.748485,0.099477,0.005583,0.111578,0.000,0.975,0.000,0.000,0.025,1,1263973674
9,0.250000,0.250000,0.250000,0.250000,0.000000,0.200000,0.200000,0.200000,0.200000,0.200000,0.000,0.000,0.000,1.000,0.000,3,1004254451601944576


## Metamodel training

In [61]:
# stack_df = pd.read_csv('data/full/logreg_train.csv').drop(columns=['Unnamed: 0'])

In [62]:
lr = LogisticRegression(penalty='l2', solver='lbfgs', max_iter=50000, class_weight='balanced')

In [63]:
X_train, X_val, y_train, y_val = train_test_split(rf_df.drop(columns=['target']), rf_df['target'], test_size=0.2, random_state=42)

In [64]:
lr.fit(X_train.drop(columns=['user_id']), y_train)

LogisticRegression(C=1.0, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=50000,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
          solver='lbfgs', tol=0.0001, verbose=0, warm_start=False)

In [65]:
cv = cross_validate(lr, stack_df.drop(columns=['user_id', 'target']), stack_df['target'], cv=10, scoring='f1_macro')

In [66]:
cv['test_score'].mean()

0.9697595083818902

0.9672409308682252 con bot prob

In [67]:
lr.fit(stack_df.drop(columns=['user_id', 'target']), stack_df['target'])

LogisticRegression(C=1.0, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=50000,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
          solver='lbfgs', tol=0.0001, verbose=0, warm_start=False)

In [102]:
z = lr.predict_proba(stack_df.drop(columns=['user_id', 'target']).iloc[1:2])

In [68]:
pickle.dump( lr, open( "../scripts/lr.model", "wb" ) )

In [194]:
z = z.reshape(xx0.shape)
plt.figure(1, figsize=(12, 9))
plt.pcolormesh(xx0, xx1, z, cmap=background_cmap)

font = {'family' : 'sans', 'size'   : 32}
plt.rc('font', **font)

for i, color in zip(logistic.classes_, colors):
    idx = np.where(y == i)
    plt.scatter(x[idx, 0], x[idx, 1], c=color) #, cmap=plt.cm.Pastel2)

plt.xlabel('Sepal length')
plt.ylabel('Sepal width')

plt.xlim(xx0.min(), xx0.max())
plt.ylim(xx1.min(), xx1.max())
plt.xticks(())
plt.yticks(())


### plot also the planes
coef = logistic.coef_
intercept = logistic.intercept_

def plot_hyperplane(c, color):
    def line(x0):
        return (-(x0 * coef[c, 0]) - intercept[c]) / coef[c, 1]
    plt.plot([x0_min, x0_max], [line(x0_min), line(x0_max)],
             ls="--", lw=4, color=color)

# colors = "rgb"
for i, color in zip(logistic.classes_, colors):
    plot_hyperplane(i, color)

plt.show()

NameError: name 'xx0' is not defined